In [1]:
import sys, os
sys.path.append("../..")
import numpy as np
from scipy.constants import h, c, eV, nano, milli, degree, pi
import pyoptix
from pyoptix.classes import (Beamline, GaussianSource, OpticalElement,
                             ToroidalMirror, PlaneFilm)
from pyoptix.optimize import focus

hc = h*c/eV
pyoptix.output_notebook()

intialzing SR library
OptiX library initialized


Loading BokehJS ...

In [2]:
lambda_laser = 1080e-9 # m
def harmonique(N=21):
    return lambda_laser/N

# Definition des optiques 

In [3]:
helimag = Beamline(name="Helimag")
source = GaussianSource(name="Generation")
m1T = ToroidalMirror(name="M1")
m2T = ToroidalMirror(name="M2")
foc = PlaneFilm(name="Focalisation")
focM1 = PlaneFilm(name="FocalisationM1")
spectro = PlaneFilm(name="Spectrometre")

In [4]:
helimag.chains["source"] = [source, focM1]
helimag.chains["M1"] = [source, m1T, focM1]
helimag.chains["Complete"] = [source, m1T, m2T, foc, spectro]

# Definition des paramètres optiques

## Source 

In [5]:
source.nrays = 1000
source.sigma_x = 1.e-4 # 100µm
source.sigma_y = 1.e-4 # 100µm
source.sigma_x_div = 1.e-3 # 1 mrad
source.sigma_y_div = 1.e-3 # 1 mrad
source.next = m1T

In [6]:
# Calcul approximatif des rayons de courbures, on prend ftot = fm1/2
p = 1.80
q = p/10
ftot = 1/(1/p+1/q)
dist_miroirs = 20e-3 #m
f_roots = np.roots([1, -2*ftot, dist_miroirs*ftot])
f_m = f_roots[f_roots>0][0]
theta = 20*degree #rad
r = f_m*np.sin(theta)/2.
R = f_m/(2*np.sin(theta))
print(p, q, ftot, f_m, r, R)

1.8 0.18 0.16363636363636364 0.3169469380642385 0.05420111859168135 0.46334542606521906


In [7]:
m1T.distance_from_previous = 1.80 # m
m1T.theta = theta # rad
m1T.phi = 90*degree # rad
m1T.minor_curvature = 1/r # m-1
m1T.major_curvature = 1/R # m-1
m1T.next = m2T

In [8]:
m2T.distance_from_previous = dist_miroirs # m
m2T.theta = theta # rad
m2T.phi = 0*degree # rad
m2T.minor_curvature = 1/r # m-1
m2T.major_curvature = 1/R # m-1
m2T.next = foc

In [9]:
foc.distance_from_previous = 40e-3
foc.phi = -90*degree #rad
foc.next = spectro
foc.set_recording("input")

In [20]:
focM1.distance_from_previous = 82.8e-3
focM1.phi = -90*degree #rad
focM1.set_recording("input")

In [11]:
spectro.distance_from_previous = 50e-3
spectro.set_recording("input")

In [12]:
helimag.active_chain = "M1"

Chaîne M1:
	Generation -> M1 -> FocalisationM1 


In [13]:
helimag.clear_impacts(clear_source=True)
helimag.align(harmonique(21))
helimag.generate(harmonique(21))
helimag.radiate()

1

In [21]:
from pyoptix.optimize import slider_optimizer
slider_optimizer(focM1, "distance_from_previous", (0.08, 0.10), .001, screen=focM1, beamline=helimag, 
                 wavelength=harmonique(21), display="yyp", light_spd=True, nrays=1000)

Helimag M1


interactive(children=(FloatSlider(value=0.0828, continuous_update=False, description='distance_from_previous',…

In [15]:
from pyoptix.optimize import slider_optimizer
slider_optimizer(m1T, "major_curvature", (0.1, 1), .1, screen=focM1, beamline=helimag, 
                 wavelength=harmonique(21), display="xxp", light_spd=True, nrays=1000)

Helimag M1


interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='major_curvature', layout=La…

In [16]:
focM1.get_diagram(1000)

,X,Y,dX,dY,Lambda
0,-0.000864,-8.908742e-05,-0.002295,-0.035888,5.142857e-08
1,0.000450,-2.945778e-05,0.002153,-0.012687,5.142857e-08
2,0.000287,-1.045266e-04,0.003033,-0.040704,5.142857e-08
3,-0.000501,8.430027e-05,-0.001224,0.030401,5.142857e-08
4,-0.002178,4.284470e-05,-0.009118,0.016347,5.142857e-08
...,...,...,...,...,...
995,0.001500,-1.265217e-07,0.006507,0.000269,5.142857e-08
996,-0.002088,2.922464e-05,-0.008930,0.012261,5.142857e-08
997,0.003375,3.377290e-05,0.014781,0.011103,5.142857e-08
998,-0.001845,2.255685e-07,-0.008072,-0.001446,5.142857e-08


In [17]:
focM1.show_diagram(1000, display="all", orthonorm=False)

None None


(ColumnDataSource(id='1881', ...),
  <bokeh.io.notebook.CommsHandle at 0x28a2df85580>])

In [18]:
f"{np.sqrt((1e-4)**2+(60*1e-3)**2)*2.35:.3e}"

'1.410e-01'

In [19]:
np.roots([1, f, -20e-3*f])

NameError: name 'f' is not defined